In [1]:
from pre import complete_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
complete_data.loc[complete_data["class"] ==  " <=50K.","class"]= " <=50K"
complete_data.loc[complete_data["class"] ==  " >50K.","class"]= " >50K"

In [3]:
complete_data = pd.concat([complete_data,pd.get_dummies(complete_data["class"],prefix = "class")],axis=1)

In [4]:
complete_data.columns.values

array(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'class', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Never-worked', 'workclass_ Private',
       'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc',
       'workclass_ State-gov', 'workclass_ Without-pay',
       'education_ 11th', 'education_ 12th', 'education_ 1st-4th',
       'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th',
       'education_ Assoc-acdm', 'education_ Assoc-voc',
       'education_ Bachelors', 'education_ Doctorate',
       'education_ HS-grad', 'education_ Masters', 'education_ Preschool',
       'education_ Prof-school', 'education_ Some-college',
       'marital-status_ Married-AF-spouse',
       'marital-status_ Married-civ-spouse',
       'marital-status_ Married-spouse-absent',
       'marital-status_ N

In [5]:
batch = 100
H = 30
learning_rate = 0.01
epochs = 30

graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,shape=[None,complete_data.shape[1]-11]) # remove "class" and "Y"
    y = tf.placeholder(tf.float32,shape = [None,2])
    W_1 = tf.Variable(tf.random_normal([complete_data.shape[1]-11,H],0,0.1))
    b_1 = tf.Variable(tf.zeros([H]))
    
    W_2 = tf.Variable(tf.random_normal([H,2],0,0.1))
    b_2 = tf.Variable(tf.zeros([2]))
    
    temp_1 = tf.nn.relu(tf.matmul(x,W_1) + b_1)
    temp_2 = tf.nn.softmax(tf.matmul(temp_1,W_2) + b_2)
    
    # loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(temp_2), reduction_indices=1))
    loss = tf.losses.mean_squared_error(temp_2,y)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    init = tf.global_variables_initializer()
    
sess = tf.InteractiveSession(graph=graph)
sess.run(init)
print("Initialized")

Initialized


In [6]:
index = 0
epoch = 0
def generate_batch(batch_size):
    global index
    global epoch
    batch_size -=1
    a = complete_data.loc["train"].drop(list(complete_data.loc["train"].select_dtypes(include="object").dtypes.index)+["class_ <=50K","class_ >50K"], axis=1)
    if index + batch_size < complete_data.loc["train"].shape[0]:
        ori_index = index
        index += batch_size
        return a.loc[ori_index:index,],complete_data.loc["train"].loc[ori_index:index,["class_ <=50K","class_ >50K"]]
    else:
        epoch +=1
        ori_index = index
        index += batch_size
        index -= complete_data.loc["train"].shape[0]
        return pd.concat([a.loc[ori_index:,],a.loc[:index,]]),pd.concat([complete_data.loc["train"].loc[ori_index:,["class_ <=50K","class_ >50K"]],complete_data.loc["train"].loc[:index,["class_ <=50K","class_ >50K"]]])

In [ ]:
step = 0 
while epoch<epochs:
    step+=1
    batch_x,batch_y = generate_batch(batch)
    feed_dict={x:batch_x,y:batch_y}
    _,loss_val = sess.run([optimizer,loss],feed_dict=feed_dict)
    if step % 50 == 0:
        print("Step: %i, Epoch: %i, Loss:%f"%(step,epoch,loss_val))
 

In [ ]:
complete_data.loc["train"].drop(list(complete_data.loc["train"].select_dtypes(include="object").dtypes.index)+["class_ <=50K","class_ >50K"], axis=1).loc[:10,]

In [ ]:
complete_data.loc["train"].loc[:10,["class_ <=50K","class_ >50K"]]